<h1>Оглавление<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Data-loader" data-toc-modified-id="Data-loader-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Data loader</a></span></li></ul></div>

# Подключение библиотек

In [1]:
import fiftyone as fo
import fiftyone.zoo as foz

# Download and load the validation split of COCO-2017
fo_dataset = foz.load_zoo_dataset("coco-2017", split="validation",
                               dataset_dir='/mnt/heap')

Found annotations at '/mnt/heap/raw/instances_val2017.json'
Images already downloaded
Existing download of split 'validation' is sufficient
Loading 'coco-2017' split 'validation'
 100% |███████████████| 5000/5000 [22.6s elapsed, 0s remaining, 243.9 samples/s]      
Dataset 'coco-2017-validation' created


session = fo.launch_app(dataset)

In [2]:
fo_dataset.compute_metadata()

## Data set

In [3]:
import torch
import fiftyone.utils.coco as fouc
from PIL import Image


class FiftyOneTorchDataset(torch.utils.data.Dataset):
    """A class to construct a PyTorch dataset from a FiftyOne dataset.
    
    Args:
        fiftyone_dataset: a FiftyOne dataset or view that will be used for training or testing
        transforms (None): a list of PyTorch transforms to apply to images and targets when loading
        gt_field ("ground_truth"): the name of the field in fiftyone_dataset that contains the 
            desired labels to load
        classes (None): a list of class strings that are used to define the mapping between
            class names and indices. If None, it will use all classes present in the given fiftyone_dataset.
    """

    def __init__(
        self,
        fiftyone_dataset,
        transforms=None,
        gt_field="ground_truth",
        classes=None,
    ):
        self.samples = fiftyone_dataset
        self.transforms = transforms
        self.gt_field = gt_field

        self.img_paths = self.samples.values("filepath")

        self.classes = classes
        if not self.classes:
            # Get list of distinct labels that exist in the view
            self.classes = self.samples.distinct(
                "%s.detections.label" % gt_field
            )

        if self.classes[0] != "background":
            self.classes = ["background"] + self.classes

        self.labels_map_rev = {c: i for i, c in enumerate(self.classes)}

    def __getitem__(self, idx):
        img_path = self.img_paths[idx]
        sample = self.samples[img_path]
        metadata = sample.metadata
        img = Image.open(img_path).convert("RGB")

        boxes = []
        detections = sample[self.gt_field].detections
        img_has_person = 0
        for det in detections:
            category_id = self.labels_map_rev[det.label]
            coco_obj = fouc.COCOObject.from_detection(
                det, metadata, category_id=category_id,
            )
            x, y, w, h = coco_obj.bbox
            
            if det.label == 'person':
                boxes.append([x, y, x + w, y + h])
                img_has_person = 1

        target = {}
        
        print(boxes)
        print('==================================')
        box = sorted(boxes)[0]  
        print(box)
        target["box"] = torch.as_tensor(box, dtype=torch.float32)                              
        target["img_has_person"] = img_has_person

        if self.transforms is not None:
            img = self.transforms(img)

        return img, target

    def __len__(self):
        return len(self.img_paths)

    def get_classes(self):
        return self.classes

In [4]:
from fiftyone import ViewField as F

person_list = ["person"]
person_view = fo_dataset.filter_labels("ground_truth",
        F("label").is_in(person_list))

print(len(person_view))

2693


## Data loader

In [5]:
import torchvision.transforms as T

img_size = (256, 256)
train_transforms = T.Compose([T.Resize(img_size),
                              T.ToTensor()])
test_transforms = T.Compose([T.Resize(img_size),
                             T.ToTensor()])

In [6]:
# split the dataset in train and test set
train_view = person_view.take(len(person_view) * 0.75, seed=51)
test_view = person_view.exclude([s.id for s in train_view])

In [7]:
torch_dataset = FiftyOneTorchDataset(train_view, train_transforms,
        classes=person_list)
torch_dataset_test = FiftyOneTorchDataset(test_view, test_transforms, 
        classes=person_list)

In [8]:
len(torch_dataset)

2019

In [9]:
len(torch_dataset_test)

674

In [10]:
# define training and validation data loaders
data_loader = torch.utils.data.DataLoader(
        torch_dataset, batch_size=5)
    
data_loader_test = torch.utils.data.DataLoader(
        torch_dataset_test, batch_size=1, shuffle=False)

# Train loop

In [11]:
from itertools import product
from pytorch_helper import PyTorchHelper

In [12]:
loss_function_xy = torch.nn.MSELoss()
loss_function_bce = torch.nn.CrossEntropyLoss()

In [13]:
def loss_function(x,y):
    return loss_function_bce(x[0], y['img_has_person']) + loss_function_xy(x[1:], y['boxes'])

In [14]:
def find_hyperparameters(config):
    learning_rates = [1e-1]
    anneal_coeff = 0.2
    anneal_epochs = [5]
    regs = config['regs']
    optimizers = config['optimizers']

    batch_size = 64
    epoch_num = 10

    run_record = {} 
    
    helper = PyTorchHelper(8,  None)

    for lr, reg, anneal_epoch, optimizer in product(learning_rates, regs, anneal_epochs, optimizers):
        lenet_model = config['model']() # base_lenet()

        loss = loss_function

        optimizer = optimizer(lenet_model.parameters(), lr=lr, weight_decay=reg)
        scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=anneal_epoch, gamma=anneal_coeff)

        loss_history = helper.train_model(config['model_name'],lenet_model, data_loader, data_loader_test, loss, optimizer, epoch_num, scheduler)

    return loss_history

In [15]:
import torch
import torch.nn as nn
import torch.optim as optim

from torchvision.models import resnet34

In [16]:
def net():
    model = resnet34(pretrained=True)
    model.fc = nn.Linear(in_features=512, out_features=5)
    return model

In [17]:
config = {
    'regs': [0.0001],
    'optimizers': [optim.SGD],
    'model': net,
    'model_name': 'best_lenet'    
}

In [18]:
run_record, best_result = find_hyperparameters(config)

[[250.82, 168.26, 320.93, 233.14], [285.55, 370.56, 297.62, 389.77]]
[250.82, 168.26, 320.93, 233.14]
[[146.68, 238.89, 247.73000000000002, 312.77]]
[146.68, 238.89, 247.73000000000002, 312.77]
[[289.54, 239.99, 355.39, 403.5]]
[289.54, 239.99, 355.39, 403.5]
[[40.0, 40.8, 273.6, 434.44], [10.07, 38.14, 57.050000000000004, 114.2], [97.86000000000001, 36.02, 148.52, 125.61000000000001], [139.64, 46.73, 192.57, 131.19], [0.49, 82.36, 71.6, 210.62], [220.39, 48.18, 272.34, 131.1], [271.23, 54.01, 320.34000000000003, 130.41], [319.21, 41.46, 373.04999999999995, 130.91], [165.92, 6.73, 219.73, 75.11], [264.05, 12.23, 305.46000000000004, 99.2], [86.18, 0.0, 134.37, 67.27], [274.28, 104.76, 355.54999999999995, 248.0], [183.63, 50.66, 224.55, 117.83], [1.0, 0.0, 374.0, 249.0]]
[0.49, 82.36, 71.6, 210.62]
[[58.38, 55.91, 504.52, 386.38]]
[58.38, 55.91, 504.52, 386.38]
prediction
tensor([[ 0.8394, -0.7502, -0.2852, -0.4345,  0.0333],
        [ 0.5533,  0.1397,  1.0542, -0.6555, -1.0701],
       

RuntimeError: Boolean value of Tensor with more than one value is ambiguous